1. Load all the necessary functions

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
import requests
import torch.nn as nn
import torch.optim as optim
import random
!pip install datasets


2. import CNN/Daily Mail dataset, from https://huggingface.co/datasets/cnn_dailymail

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from datasets import load_dataset

# Modify the dataset loading to load a smaller subset
dataset = load_dataset('cnn_dailymail', '3.0.0', split="train[:500]")  # Load only 500 examples for training

# Train, validation, and test splits
train_data = dataset
val_data = load_dataset('cnn_dailymail', '3.0.0', split="validation[:100]")  # Load 100 examples for validation
test_data = load_dataset('cnn_dailymail', '3.0.0', split="test[:100]")  # Load 100 examples for testing

def preprocess_text(text):
    sentences = sent_tokenize(text)
    tokenized_sentences = [word_tokenize(sent) for sent in sentences]
    return tokenized_sentences

# Extract summaries and texts from each split
train_summaries = [example['highlights'] for example in train_data]
train_texts = [example['article'] for example in train_data]

val_summaries = [example['highlights'] for example in val_data]
val_texts = [example['article'] for example in val_data]

test_summaries = [example['highlights'] for example in test_data]
test_texts = [example['article'] for example in test_data]

# Print the number of examples in each split
print("Data Dimensions:")
print("Train data examples:", len(train_data))
print("Validation data examples:", len(val_data))
print("Test data examples:", len(test_data))

# Print the dimensions of target summaries and texts (articles)
print("\nTarget Summaries Dimensions:")
print("Train summaries:", len(train_summaries))
print("Validation summaries:", len(val_summaries))
print("Test summaries:", len(test_summaries))

print("\nTexts (Articles) Dimensions:")
print("Train texts:", len(train_texts))
print("Validation texts:", len(val_texts))
print("Test texts:", len(test_texts))


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import torch.nn.utils.rnn as rnn_utils


vocab_input = {'<OOV>': 0}
word_counter_input = Counter(word for text in train_texts for word in nltk.word_tokenize(text))
for word, count in word_counter_input.items():
    if word not in vocab_input:
        vocab_input[word] = len(vocab_input)

# Create a vocabulary from the training set for target summaries
vocab_target = {'<OOV>': 0}
word_counter_target = Counter(word for summary in train_summaries for word in nltk.word_tokenize(summary))
for word, count in word_counter_target.items():
    if word not in vocab_target:
        vocab_target[word] = len(vocab_target)

# Function to convert a sentence to a sequence of integers for input text
def sentence_to_sequence_input(sentence):
    return [vocab_input[word] if word in vocab_input else vocab_input['<OOV>'] for word in sentence]

# Function to convert a sentence to a sequence of integers for target summaries
def sentence_to_sequence_target(sentence):
    return [vocab_target[word] if word in vocab_target else vocab_target['<OOV>'] for word in sentence]

# Convert the sentences in each split to sequences of integers for input text
train_sequences_input = [sentence_to_sequence_input(text) for text in train_texts]
val_sequences_input = [sentence_to_sequence_input(text) for text in val_texts]
test_sequences_input = [sentence_to_sequence_input(text) for text in test_texts]

# Convert the target summaries in each split to sequences of integers
train_sequences_target = [sentence_to_sequence_target(summary) for summary in train_summaries]
val_sequences_target = [sentence_to_sequence_target(summary) for summary in val_summaries]
test_sequences_target = [sentence_to_sequence_target(summary) for summary in test_summaries]

# Maximum length for each sentence
max_length = 512
max_summary_length = 100  # Set the desired maximum length for target summaries

train_padded_input = rnn_utils.pad_sequence([torch.tensor(seq) for seq in train_sequences_input], batch_first=True, padding_value=0)
val_padded_input = rnn_utils.pad_sequence([torch.tensor(seq) for seq in val_sequences_input], batch_first=True, padding_value=0)
test_padded_input = rnn_utils.pad_sequence([torch.tensor(seq) for seq in test_sequences_input], batch_first=True, padding_value=0)

# Pad the sequences for target summaries
train_padded_target = rnn_utils.pad_sequence([torch.tensor(seq) for seq in train_sequences_target], batch_first=True, padding_value=0)
val_padded_target = rnn_utils.pad_sequence([torch.tensor(seq) for seq in val_sequences_target], batch_first=True, padding_value=0)
test_padded_target = rnn_utils.pad_sequence([torch.tensor(seq) for seq in test_sequences_target], batch_first=True, padding_value=0)

import torch
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, sequences_input, sequences_target, max_length):
        self.sequences_input = sequences_input
        self.sequences_target = sequences_target
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences_input)

    def __getitem__(self, index):
        # Get the input sequence and target summary at the given index
        input_sequence = self.sequences_input[index]
        target_summary = self.sequences_target[index]

        # Pad the sequences to the maximum length
        input_sequence = self.pad_sequence(input_sequence, self.max_length)
        target_summary = self.pad_sequence(target_summary, self.max_length)

        # Convert the sequences to tensors
        input_tensor = torch.tensor(input_sequence, dtype=torch.long)
        target_tensor = torch.tensor(target_summary, dtype=torch.long)

        return input_tensor, target_tensor

    def pad_sequence(self, sequence, max_length):
        if len(sequence) < max_length:
            # Pad the sequence with zeros if its length is less than max_length
            sequence = sequence + [0] * (max_length - len(sequence))
        else:
            # Truncate the sequence if its length exceeds max_length
            sequence = sequence[:max_length]
        return sequence

# Create custom datasets
train_dataset = CustomDataset(train_sequences_input, train_sequences_target, max_length)
val_dataset = CustomDataset(val_sequences_input, val_sequences_target, max_length)
test_dataset = CustomDataset(test_sequences_input, test_sequences_target, max_length)

# Define batch size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Manually retrieve a batch of data from the train_loader
for batch_inputs, batch_targets in train_loader:
    # Print the dimensions of the tensors in the batch
    print("Batch Input Dimensions:", batch_inputs.shape)
    print("Batch Target Dimensions:", batch_targets.shape)
    break  # Exit the loop after the first batch to avoid printing all batches


# Define the Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

# Define the Decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

# Define the Seq2Seq model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Encoder forward pass
        hidden, cell = self.encoder(src)

        # Decoder initial input (start token) and hidden states
        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder_input_dim = len(vocab_input)
decoder_output_dim = len(vocab_target)
embedding_dim = 100
hidden_dim = 256
num_layers = 2
dropout = 0.5

encoder = Encoder(encoder_input_dim, embedding_dim, hidden_dim, num_layers, dropout)
decoder = Decoder(decoder_output_dim, embedding_dim, hidden_dim, num_layers, dropout)
model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import torch


# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Function to calculate accuracy
def calculate_accuracy(preds, targets):
    _, predicted = torch.max(preds, 1)
    correct = (predicted == targets).sum().item()
    return correct / len(targets)

# Function to calculate loss
def calculate_loss(preds, targets):
    loss = criterion(preds, targets)
    return loss.item()

def train(model, num_epochs, train_loader, val_loader, print_every=5):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in tqdm(range(1, num_epochs + 1)):
        # Training
        model.train()
        running_train_loss = 0.0
        total_train_samples = 0
        correct_train = 0

        for batch_inputs, batch_targets in train_loader:
            optimizer.zero_grad()
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)
            loss = criterion(outputs.view(-1, outputs.shape[-1]), batch_targets.view(-1))
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            total_train_samples += len(batch_inputs) * batch_inputs.shape[1]
            correct_train += (outputs.argmax(1) == batch_targets.view(-1)).sum().item()

        train_loss = running_train_loss / len(train_loader)
        train_losses.append(train_loss)
        train_accuracy = correct_train / total_train_samples
        train_accuracies.append(train_accuracy)

        # Validation
        model.eval()
        running_val_loss = 0.0
        total_val_samples = 0
        correct_val = 0

        with torch.no_grad():
            for batch_inputs, batch_targets in val_loader:
                outputs = model(batch_inputs)
                loss = criterion(outputs.view(-1, outputs.shape[-1]), batch_targets.view(-1))

                running_val_loss += loss.item()
                total_val_samples += len(batch_inputs) * batch_inputs.shape[1]
                correct_val += (outputs.argmax(1) == batch_targets.view(-1)).sum().item()

        val_loss = running_val_loss / len(val_loader)
        val_losses.append(val_loss)
        val_accuracy = correct_val / total_val_samples
        val_accuracies.append(val_accuracy)

        # Print and plot results every 'print_every' epochs
        if epoch % print_every == 0:
            print(f"\nEpoch {epoch}/{num_epochs}")
            print(f"Training Loss: {train_loss:.4f} - Training Accuracy: {train_accuracy:.4f}")
            print(f"Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")

            # Plot graphs
            if epoch == num_epochs or epoch % (5 * print_every) == 0:
                plt.figure(figsize=(12, 6))
                plt.subplot(1, 2, 1)
                plt.plot(np.arange(1, epoch + 1), train_losses, label='Training Loss')
                plt.plot(np.arange(1, epoch + 1), val_losses, label='Validation Loss')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()

                plt.subplot(1, 2, 2)
                plt.plot(np.arange(1, epoch + 1), train_accuracies, label='Training Accuracy')
                plt.plot(np.arange(1, epoch + 1), val_accuracies, label='Validation Accuracy')
                plt.xlabel('Epoch')
                plt.ylabel('Accuracy')
                plt.legend()

                plt.show()

    return train_losses, val_losses, train_accuracies, val_accuracies




In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import requests

nltk.download('punkt')
nltk.download('stopwords')

# Preprocessing functions

def preprocess_text(text):
    words = word_tokenize(text)
    table = str.maketrans('', '', string.punctuation)
    words = [word.translate(table) for word in words]
    words = [word.lower() for word in words]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if len(word) > 1]
    return words

def pad_or_truncate_text(sentence, max_length):
    if len(sentence) < max_length:
        sentence = sentence + ['<PAD>'] * (max_length - len(sentence))
    else:
        sentence = sentence[:max_length]
    return sentence

def preprocess_target_summary(summary, max_length):
    return pad_or_truncate_text(preprocess_text(summary), max_length)

# ... (Previous code remains unchanged)

# Fetch train data and target summaries
train_data = fetch_data(TRAIN_API_URL)
train_summaries = [example['row']['highlights'] for example in train_data['rows']]
train_texts = [example['row']['article'] for example in train_data['rows']]

# Fetch val data and target summaries
val_data = fetch_data(VAL_API_URL)
val_summaries = [example['row']['highlights'] for example in val_data['rows']]
val_texts = [example['row']['article'] for example in val_data['rows']]

# Fetch test data and target summaries
test_data = fetch_data(TEST_API_URL)
test_summaries = [example['row']['highlights'] for example in test_data['rows']]
test_texts = [example['row']['article'] for example in test_data['rows']]

# Preprocess article texts and target summaries for each split
max_length = 512
train_texts = [pad_or_truncate_text(preprocess_text(article_text), max_length) for article_text in train_texts]
val_texts = [pad_or_truncate_text(preprocess_text(article_text), max_length) for article_text in val_texts]
test_texts = [pad_or_truncate_text(preprocess_text(article_text), max_length) for article_text in test_texts]
train_targets = [preprocess_target_summary(summary, max_length) for summary in train_summaries]
val_targets = [preprocess_target_summary(summary, max_length) for summary in val_summaries]
test_targets = [preprocess_target_summary(summary, max_length) for summary in test_summaries]


# Print preprocessed data
print("Train Texts:", train_texts)
print("Train Targets:", train_targets)
print("Val Texts:", val_texts)
print("Val Targets:", val_targets)
print("Test Texts:", test_texts)
print("Test Targets:", test_targets)


print("Train Texts Dimensions:")
for i, text in enumerate(train_texts):
    print(f"Text {i + 1} length:", len(text))

print("\nTrain Targets Dimensions:")
for i, summary in enumerate(train_targets):
    print(f"Summary {i + 1} length:", len(summary))

print("\nVal Texts Dimensions:")
for i, text in enumerate(val_texts):
    print(f"Text {i + 1} length:", len(text))

print("\nVal Targets Dimensions:")
for i, summary in enumerate(val_targets):
    print(f"Summary {i + 1} length:", len(summary))

print("\nTest Texts Dimensions:")
for i, text in enumerate(test_texts):
    print(f"Text {i + 1} length:", len(text))

print("\nTest Targets Dimensions:")
for i, summary in enumerate(test_targets):
    print(f"Summary {i + 1} length:", len(summary))

In [ ]:
# Represent text as word embeddings

# -Word embeddings examples are Word2Vec, GloVe, FastText, and represent each word as a dense vector.
# follow https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html.

import torch
import torch.nn as nn
import torch.optim as optim

embedding_dim = 256
hidden_dim = 256
torch.manual_seed(1)

# List of train texts
train_preprocessed_text = []
for article_text in train_texts:  # Use train_texts instead of looping through train_data
    train_preprocessed_text.append(pad_or_truncate_text(article_text, max_length))

# List of val texts
val_preprocessed_text = []
for article_text in val_texts:  # Use val_texts instead of looping through val_data
    val_preprocessed_text.append(pad_or_truncate_text(article_text, max_length))

# List of test texts
test_preprocessed_text = []
for article_text in test_texts:  # Use test_texts instead of looping through test_data
    test_preprocessed_text.append(pad_or_truncate_text(article_text, max_length))


# Build vocabulary from both texts and summaries
all_texts_and_summaries = train_preprocessed_text + val_preprocessed_text + test_preprocessed_text + train_summaries + val_summaries + test_summaries
word_freq = {}  # A dictionary to store word frequencies

for sentence in all_texts_and_summaries:
    for word in sentence:
        if word not in word_freq:
            word_freq[word] = 0
        word_freq[word] += 1

# Sort words based on frequency in descending order
sorted_words = sorted(word_freq.keys(), key=lambda x: word_freq[x], reverse=True)

# Take only the top vocab_size - 1 words, and add an OOV token
vocab_size = len(sorted_words)
sorted_words = sorted_words[:vocab_size - 1] + ['<OOV>', '<SOS>']

# Build word_to_num based on the sorted_words list
word_to_num = {word: i for i, word in enumerate(sorted_words)}
word_to_num['<OOV>'] = word_to_num.get('<OOV>', vocab_size - 2)  # The second to last index


def sentence_to_indices(sentence):
    return [word_to_num.get(word, word_to_num['<OOV>']) for word in sentence]

class wordembedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(wordembedding, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.vocab_size = vocab_size

    def forward(self, inputs):
        return self.embeddings(inputs)

# Create the model and the optimizer
word_model = wordembedding(len(sorted_words), embedding_dim)  # Use len(sorted_words) as vocab_size
optimizer = optim.SGD(word_model.parameters(), lr=0.001)




# ... (previous code remains unchanged)

# Loop over train_targets and generate embeddings for summaries
train_summary_embeddings_list = []
for summary in train_targets:
    summary_idxs = [word_to_num.get(word, word_to_num['<OOV>']) for word in summary]  # Get the index of each word in the summary
    summary_idxs = torch.tensor(summary_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(summary_idxs)
    train_summary_embeddings_list.append(embeddings)  # Store the embeddings

# Loop over val_targets and generate embeddings for summaries
val_summary_embeddings_list = []
for summary in val_targets:
    summary_idxs = [word_to_num.get(word, word_to_num['<OOV>']) for word in summary]  # Get the index of each word in the summary
    summary_idxs = torch.tensor(summary_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(summary_idxs)
    val_summary_embeddings_list.append(embeddings)  # Store the embeddings

# Loop over test_targets and generate embeddings for summaries
test_summary_embeddings_list = []
for summary in test_targets:
    summary_idxs = [word_to_num.get(word, word_to_num['<OOV>']) for word in summary]  # Get the index of each word in the summary
    summary_idxs = torch.tensor(summary_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(summary_idxs)
    test_summary_embeddings_list.append(embeddings)  # Store the embeddings

train_summary_embeddings_tensor = torch.stack(train_summary_embeddings_list)
val_summary_embeddings_tensor = torch.stack(val_summary_embeddings_list)
test_summary_embeddings_tensor = torch.stack(test_summary_embeddings_list)

train_text_embeddings_list = []
val_text_embeddings_list = []
test_text_embeddings_list = []

# Loop over train_preprocessed_text and generate embeddings for texts
for sentence in train_preprocessed_text:
    sentence_idxs = [word_to_num[word] for word in sentence]  # Get the index of each word in the sentence
    sentence_idxs = torch.tensor(sentence_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(sentence_idxs)
    train_text_embeddings_list.append(embeddings)  # Store the embeddings

# Loop over val_preprocessed_text and generate embeddings for texts
for sentence in val_preprocessed_text:
    sentence_idxs = [word_to_num[word] for word in sentence]  # Get the index of each word in the sentence
    sentence_idxs = torch.tensor(sentence_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(sentence_idxs)
    val_text_embeddings_list.append(embeddings)  # Store the embeddings

# Loop over test_preprocessed_text and generate embeddings for texts
for sentence in test_preprocessed_text:
    sentence_idxs = [word_to_num[word] for word in sentence]  # Get the index of each word in the sentence
    sentence_idxs = torch.tensor(sentence_idxs, dtype=torch.long)  # Convert the list to tensor outside the loop
    embeddings = word_model(sentence_idxs)
    test_text_embeddings_list.append(embeddings)  # Store the embeddings

# Convert lists to PyTorch tensors for texts
train_text_embeddings_tensor = torch.stack(train_text_embeddings_list).to(torch.long)
val_text_embeddings_tensor = torch.stack(val_text_embeddings_list).to(torch.long)
test_text_embeddings_tensor = torch.stack(test_text_embeddings_list).to(torch.long)


# print embeddings
print("Train Summary Embeddings:")
print(train_summary_embeddings_tensor)

print("Validation Summary Embeddings:")
print(val_summary_embeddings_tensor)

print("Test Summary Embeddings:")
print(test_summary_embeddings_tensor)

print("Train Text Embeddings:")
print(train_text_embeddings_tensor)

print("Validation Text Embeddings:")
print(val_text_embeddings_tensor)

print("Test Text Embeddings:")
print(test_text_embeddings_tensor)

train_summary_embeddings = train_summary_embeddings_tensor
val_summary_embeddings = val_summary_embeddings_tensor
test_summary_embeddings = test_summary_embeddings_tensor
train_text_embeddings = train_text_embeddings_tensor
val_text_embeddings = val_text_embeddings_tensor
test_text_embeddings = test_text_embeddings_tensor


print(train_text_embeddings.size())
print(train_summary_embeddings.size())
print(val_text_embeddings.size())
print(val_summary_embeddings.size())
print(test_text_embeddings.size())
print(test_summary_embeddings.size())
print("Shape of train_text_embeddings_tensor:", train_text_embeddings_tensor.shape)
print("Shape of train_summary_embeddings_tensor:", train_summary_embeddings_tensor.shape)
print("Shape of val_text_embeddings_tensor:", val_text_embeddings_tensor.shape)
print("Shape of val_summary_embeddings_tensor:", val_summary_embeddings_tensor.shape)
print("Shape of test_text_embeddings_tensor:", test_text_embeddings_tensor.shape)
print("Shape of test_summary_embeddings_tensor:", test_summary_embeddings_tensor.shape)







In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Assuming you have the following variables containing the embeddings:
train_text_embeddings_tensor, train_summary_embeddings_tensor
val_text_embeddings_tensor, val_summary_embeddings_tensor
test_text_embeddings_tensor, test_summary_embeddings_tensor

# Define a custom dataset for your training data
class CustomDataset(Dataset):
    def __init__(self, text_embeddings, summary_embeddings):
        self.text_embeddings = text_embeddings
        self.summary_embeddings = summary_embeddings

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        return self.text_embeddings[idx], self.summary_embeddings[idx]

# Batch size for training, validation, and test data loaders
batch_size = 32

# Create instances of the custom dataset for your training, validation, and test data
train_dataset = CustomDataset(train_text_embeddings_tensor, train_summary_embeddings_tensor)
val_dataset = CustomDataset(val_text_embeddings_tensor, val_summary_embeddings_tensor)
test_dataset = CustomDataset(test_text_embeddings_tensor, test_summary_embeddings_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)


print("Train Loader Dimensions:")
for inputs, targets in train_loader:
    print("Input Tensor Dimensions:", inputs.size())
    print("Target Tensor Dimensions:", targets.size())
    break  # Print dimensions for only the first batch

print("\nValidation Loader Dimensions:")
for inputs, targets in val_loader:
    print("Input Tensor Dimensions:", inputs.size())
    print("Target Tensor Dimensions:", targets.size())
    break  # Print dimensions for only the first batch

print("\nTest Loader Dimensions:")
for inputs, targets in test_loader:
    print("Input Tensor Dimensions:", inputs.size())
    print("Target Tensor Dimensions:", targets.size())
    break  # Print dimensions for only the first batch

print("Train Loader Number of Samples:", len(train_loader.dataset))
print("Validation Loader Number of Samples:", len(val_loader.dataset))
print("Test Loader Number of Samples:", len(test_loader.dataset))


In [ ]:
########  NN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib as plt

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


class Encoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, embedded_input):
        encoder_outputs, (hidden_state, cell_state) = self.rnn(embedded_input)
        return encoder_outputs, (hidden_state, cell_state)

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))

    def forward(self, hidden_tuple, encoder_outputs):
        hidden, _ = hidden_tuple
        max_len = encoder_outputs.size(1)
        h = hidden.repeat(max_len, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(1, 2)
        attn_scores = F.softmax(torch.bmm(h, self.attn(encoder_outputs)), dim=2)
        context = torch.bmm(attn_scores, encoder_outputs)
        return context


class Decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(Decoder, self).__init__()
        self.rnn = nn.LSTM(embedding_dim + hidden_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, embedded_target, hidden, encoder_outputs):
        context = self.attention(hidden, encoder_outputs)
        rnn_input = torch.cat((embedded_target, context.unsqueeze(1).repeat(1, embedded_target.size(1), 1)), dim=2)
        decoder_outputs, hidden = self.rnn(rnn_input, hidden)
        output_sequence = F.log_softmax(self.fc(decoder_outputs), dim=2)
        return output_sequence, hidden


class Seq2SeqAttentionWithEmbedding(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, src_embedding_dim, tgt_embedding_dim, hidden_dim):
        super(Seq2SeqAttentionWithEmbedding, self).__init__()
        self.embedding_source = nn.Embedding(src_vocab_size, src_embedding_dim)
        self.embedding_target = nn.Embedding(tgt_vocab_size, tgt_embedding_dim)
        self.encoder = Encoder(src_embedding_dim, hidden_dim)
        self.decoder = Decoder(tgt_embedding_dim, hidden_dim, tgt_vocab_size)
        self.attention = Attention(hidden_dim)

    def forward(self, src_input, tgt_input):
        embedded_source = self.embedding_source(src_input)
        embedded_target = self.embedding_target(tgt_input)

        encoder_outputs, hidden = self.encoder(embedded_source)
        context = self.attention(hidden, encoder_outputs)
        rnn_input = torch.cat((embedded_target, context), dim=2)

        decoder_outputs, hidden = self.decoder.rnn(rnn_input, hidden)
        output_sequence = F.log_softmax(self.decoder.fc(decoder_outputs), dim=2)
        return output_sequence, hidden



src_vocab_size = len(word_to_num)
tgt_vocab_size = len(word_to_num)  # Use the same vocabulary for both source and target for now, you can create separate ones if needed
model = Seq2SeqAttentionWithEmbedding(src_vocab_size, tgt_vocab_size, embedding_dim, embedding_dim, hidden_dim)


# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm as tqdm_module  # Importing tqdm with a different name to avoid conflicts
import matplotlib.pyplot as plt

# Training loop
num_epochs = 20

train_accuracy_list = []
val_accuracy_list = []
train_loss_list = []
val_loss_list = []

for epoch in range(num_epochs):
    word_model.train()
    total_loss = 0
    total_batches = 0
    correct_predictions = 0
    total_samples = 0

    with tqdm_module(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for batch_idx, (src_input, tgt_input) in enumerate(train_loader):
            src_input, tgt_input = src_input.to(device), tgt_input.to(device).long()

            batch_size = src_input.size(0)

            # Calculate max_tgt_length dynamically for the current batch
            max_tgt_length = tgt_input.size(1)

            sos_token_idx = torch.tensor([word_to_num['<SOS>']], dtype=torch.long).to(device)  # Shape: [1]

            sos_token_emb = word_model(sos_token_idx)  # Shape: [1, embedding_dim]
            sos_token_emb = sos_token_emb.repeat(batch_size, 1, 1)  # Shape: [batch_size, 1, embedding_dim]

            tgt_input_emb = word_model(tgt_input)  # Assuming tgt_input is 2D [batch_size, sequence_length]
            tgt_input_with_sos = torch.cat([sos_token_emb, tgt_input_emb], dim=1)

            optimizer.zero_grad()

            output_sequence, _ = word_model(src_input, tgt_input_with_sos[:, :-1, :])
            total_samples += tgt_input.size(0) * tgt_input.size(1)  # Calculate before reshaping tgt_input
            tgt_input = tgt_input.contiguous().view(-1).to(torch.long)

            loss = criterion(output_sequence, tgt_input)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_batches += 1

            # calculate training accuracy
            _, predicted = torch.max(output_sequence, 2)  # Use dim=2 to find max along time steps
            correct_predictions += (predicted == tgt_input.view_as(predicted)).sum().item()

            pbar.update(1)

    avg_loss = total_loss / total_batches
    train_accuracy = correct_predictions / total_samples

    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(avg_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    # Validation
    word_model.eval()
    total_loss = 0
    total_batches = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch_idx, (src_input, tgt_input) in enumerate(val_loader):
            src_input, tgt_input = src_input.to(device).to(torch.long), tgt_input.to(device).to(torch.long)

            output_sequence, _ = word_model(src_input, tgt_input[:, :-1])

            output_sequence = output_sequence.contiguous().view(-1, output_sequence.size(-1))
            total_samples += tgt_input.size(0) * tgt_input.size(1)  # Calculate before reshaping tgt_input
            tgt_input = tgt_input.contiguous().view(-1).to(torch.long)  # Convert to 1D LongTensor

            loss = criterion(output_sequence, tgt_input)

            total_loss += loss.item()
            total_batches += 1

            # Calculate validation accuracy
            _, predicted = torch.max(output_sequence, 1)
            correct_predictions += (predicted == tgt_input).sum().item()

    avg_loss = total_loss / total_batches
    val_accuracy = correct_predictions / total_samples

    val_accuracy_list.append(val_accuracy)
    val_loss_list.append(avg_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {avg_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Graphs every 5 epochs
    if (epoch + 1) % 5 == 0:
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(range(1, epoch + 2), train_accuracy_list, label="Train Accuracy")
        plt.plot(range(1, epoch + 2), val_accuracy_list, label="Validation Accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.title("Training and Validation Accuracy vs. Epoch")

        plt.subplot(1, 2, 2)
        plt.plot(range(1, epoch + 2), train_loss_list, label="Train Loss")
        plt.plot(range(1, epoch + 2), val_loss_list, label="Validation Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.title("Training and Validation Loss vs. Epoch")

        plt.tight_layout()
        plt.show()

# Final graphs
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_accuracy_list, label="Train Accuracy")
plt.plot(range(1, num_epochs + 1), val_accuracy_list, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training and Validation Accuracy vs. Epoch")

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_loss_list, label="Train Loss")
plt.plot(range(1, num_epochs + 1), val_loss_list, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training and Validation Loss vs. Epoch")

plt.tight_layout()
plt.show()


Epoch 1/20:   0%|          | 0/3 [00:00<?, ?batch/s]


IndexError: ignored